# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
!pip install pandas
!pip install requests
!pip install bs4
!pip install plotly

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: Cryptograp

###  Part 1:Scraping webpage to create the dataframe 

In [15]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(url).text
soup = BeautifulSoup(data,"lxml")

In [99]:
soup.find("table")

<table cellpadding="2" cellspacing="0" rules="all" style="width:100%; border-collapse:collapse; border:1px solid #ccc;">
<tbody><tr>
<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M1A</b><br/><span style="font-size:85%;"><i>Not assigned</i></span>
</p>
</td>
<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M2A</b><br/><span style="font-size:85%;"><i>Not assigned</i></span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M3A</b><br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>)</span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M4A</b><br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>)</span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M5A</b><br/><span style="font-size:85%;"><a hr

In [34]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

toronto_df=pd.DataFrame(table_contents)
toronto_df['Borough']=toronto_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


In [35]:
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [41]:
toronto_df.shape

(103, 3)

### Part 2: Getting the latitude and longitude for each postal code

In [55]:
#using the watson interface to read the Geospatial_Coordinates in pandas 

In [56]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [60]:
coordinates=coordinates.rename(columns={"Postal Code":"PostalCode"})

In [75]:
#merging the two dataframes
neighborhoods=pd.merge(toronto_df,coordinates,on="PostalCode")

In [77]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [80]:
print('The dataframe has {} boroughs'.format(
        len(neighborhoods['Borough'].unique())
    )
)

The dataframe has 15 boroughs


### Part 3

#### Use geopy library to get the latitude and longitude values of Toronto.


In [84]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [97]:
!pip install folium
import folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


#### Create a map of Toronto with neighborhoods superimposed on top.


In [98]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood,postalcode in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood'],neighborhoods['PostalCode']):
    label = '{}, {},{}'.format(neighborhood, borough,postalcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [91]:
#We segment and cluster only the neighborhoods in Central Toronto

In [100]:
central_toronto = neighborhoods[neighborhoods['Borough'] == 'Central Toronto'].reset_index(drop=True)
central_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678


In [101]:
address = 'Central Toronto,Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.65238435, -79.38356765.


In [103]:
# create map of Central Toronto using latitude and longitude values
map_central = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood,postalcode in zip(central_toronto['Latitude'], central_toronto['Longitude'], central_toronto['Neighborhood'],central_toronto['PostalCode']):
    label = '{}, {}'.format(neighborhood,postalcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_central)  
    
map_central

#### Define Foursquare Credentials and Version


In [104]:
CLIENT_ID = 'MGDPHEGCADAVPLR5OP4JATKQMXUJYD5HQ2IQ3ZNRZJBOOBNN' # your Foursquare ID
CLIENT_SECRET = 'VEQLJJVV33S4Y110S3M0FVZC1A14RE5NDD20DDDYRSRE5BLH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MGDPHEGCADAVPLR5OP4JATKQMXUJYD5HQ2IQ3ZNRZJBOOBNN
CLIENT_SECRET:VEQLJJVV33S4Y110S3M0FVZC1A14RE5NDD20DDDYRSRE5BLH


#### Let's explore the first neighborhood in our dataframe.


In [105]:
central_toronto.loc[0, 'Neighborhood']

'Lawrence Park'

#### Get the neighborhood's latitude and longitude values.


In [106]:
neighborhood_latitude = central_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = central_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = central_toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


##### Now, let's get the top 100 venues that are in Lawrence Park within a radius of 500 meters.


In [107]:
#creating the Get request url
LIMIT=100
radius=500
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
neighborhood_latitude,
neighborhood_longitude,
radius,
LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=MGDPHEGCADAVPLR5OP4JATKQMXUJYD5HQ2IQ3ZNRZJBOOBNN&client_secret=VEQLJJVV33S4Y110S3M0FVZC1A14RE5NDD20DDDYRSRE5BLH&v=20180605&ll=43.7280205,-79.3887901&radius=500&limit=100'

In [108]:
#Send the Get request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '606de42dc190c03a95fd7f6a'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.7325205045, 'lng': -79.3825744605273},
   'sw': {'lat': 43.7235204955, 'lng': -79.3950057394727}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.72696303913755,
          'lng': -79.39438246708775}],
        'distance': 465,
        'c

In [110]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [111]:
#Clean the json and structure it into a pandas dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,HYC Design Inc.,Business Service,43.726793,-79.391681
2,Zodiac Swim School,Swim School,43.728532,-79.382860
3,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


In [112]:
#How many venues were returned from Foursquare API
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


### Let's create a function to repeat the same process to all the neighborhoods in Central Toronto

In [114]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _Central_venues_.


In [115]:
central_venues=getNearbyVenues(names=central_toronto['Neighborhood'],latitudes=central_toronto['Latitude'],longitudes=central_toronto['Longitude'])

Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
North Toronto West
The Annex, North Midtown, Yorkville
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park


#### Checking the size of the dataframe

In [127]:
print(central_venues.shape)
central_venues.head()

(103, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,HYC Design Inc.,43.726793,-79.391681,Business Service
2,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
3,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
4,Roselawn,43.711695,-79.416936,Ceiling Champions,43.713891,-79.420702,Home Service


#### How many venues were returned for each neighborhood

In [125]:
central_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,33,33,33,33,33,33
Davisville North,6,6,6,6,6,6
Forest Hill North & West,4,4,4,4,4,4
Lawrence Park,4,4,4,4,4,4
"Moore Park, Summerhill East",2,2,2,2,2,2
North Toronto West,19,19,19,19,19,19
Roselawn,2,2,2,2,2,2
"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",14,14,14,14,14,14
"The Annex, North Midtown, Yorkville",19,19,19,19,19,19


#### Let's find out how many unique categories can be curated from all the returned venues


In [128]:
print('There are {} uniques categories.'.format(len(central_venues['Venue Category'].unique())))

There are 60 uniques categories.


### Analyzing each category

In [129]:
# one hot encoding
central_onehot = pd.get_dummies(central_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
central_onehot['Neighborhood'] = central_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [central_onehot.columns[-1]] + list(central_onehot.columns[:-1])
central_onehot = central_onehot[fixed_columns]

central_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Business Service,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Department Store,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,History Museum,Home Service,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Park,Pharmacy,Pizza Place,Pub,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,Lawrence Park,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Roselawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [130]:
# Let's examine the size
central_onehot.shape

(103, 61)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [131]:
central_grouped = central_onehot.groupby('Neighborhood').mean().reset_index()
central_grouped

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Business Service,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Department Store,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,History Museum,Home Service,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Park,Pharmacy,Pizza Place,Pub,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.00,0.00,0.060606,0.000000,0.000000,0.060606,0.000000,0.090909,0.030303,0.000000,0.030303,0.000000,0.000000,0.000000,0.0,0.030303,0.030303,0.030303,0.060606,0.000000,0.000000,0.0,0.000000,0.030303,0.060606,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.030303,0.090909,0.000000,0.030303,0.000000,0.090909,0.030303,0.000000,0.000000,0.0,0.000000,0.060606,0.00,0.0,0.030303,0.030303,0.00,0.000000,0.000000
1,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.166667,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000
2,Forest Hill North & West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.00,0.0,0.000000,0.000000,0.25,0.000000,0.000000
3,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.25,0.0,0.000000,0.000000,0.00,0.000000,0.000000
4,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.00,0.5,0.000000,0.000000,0.00,0.000000,0.000000
5,North Toronto West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.052632,0.052632,0.157895,0.105263,0.000000,0.000000,0.052632,0.000000,0.000000,0.052632,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.052632,0.052632,0.000000,0.052632,0.052632,0.000000,0.000000,0.000000,0.052632,0.052632,0.000000,0.000000,0.052632,0.052632,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.052632
6,Roselawn,0.000000,0.000000,0.0000

In [132]:
central_grouped.shape

(9, 61)

#### Let's print each neighborhood along with the top 5 most common venues


In [133]:
num_top_venues = 5

for hood in central_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = central_grouped[central_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
            venue  freq
0  Sandwich Place  0.09
1    Dessert Shop  0.09
2     Pizza Place  0.09
3     Coffee Shop  0.06
4             Gym  0.06


----Davisville North----
               venue  freq
0              Hotel  0.17
1     Breakfast Spot  0.17
2   Department Store  0.17
3  Food & Drink Shop  0.17
4               Park  0.17


----Forest Hill North & West----
                 venue  freq
0                Trail  0.25
1        Jewelry Store  0.25
2     Sushi Restaurant  0.25
3                 Park  0.25
4  American Restaurant  0.00


----Lawrence Park----
              venue  freq
0          Bus Line  0.25
1  Business Service  0.25
2              Park  0.25
3       Swim School  0.25
4               Pub  0.00


----Moore Park, Summerhill East----
                 venue  freq
0         Tennis Court   0.5
1          Summer Camp   0.5
2  American Restaurant   0.0
3           Restaurant   0.0
4   Italian Restaurant   0.0


----North Toronto West----
                ve

#### Let's put that into a _pandas_ dataframe

In [134]:
#First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [135]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = central_grouped['Neighborhood']

for ind in np.arange(central_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(central_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Gym,Sushi Restaurant,Italian Restaurant,Coffee Shop,Café,Pharmacy,Gourmet Shop
1,Davisville North,Department Store,Hotel,Park,Sandwich Place,Breakfast Spot,Food & Drink Shop,Fast Food Restaurant,Diner,Donut Shop,Farmers Market
2,Forest Hill North & West,Jewelry Store,Trail,Park,Sushi Restaurant,Food & Drink Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Yoga Studio
3,Lawrence Park,Park,Swim School,Business Service,Bus Line,Yoga Studio,Food & Drink Shop,Donut Shop,Farmers Market,Fast Food Restaurant,Fried Chicken Joint
4,"Moore Park, Summerhill East",Tennis Court,Summer Camp,Yoga Studio,Department Store,Gym,Greek Restaurant,Gourmet Shop,Gas Station,Garden,Fried Chicken Joint


### Clustering Neighborhoods

In [148]:
# Run k-means ti cluster the data in 5 clusters

# set number of clusters
kclusters = 5

central_grouped_clustering = central_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(central_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 2, 0, 4, 1, 3, 1, 1], dtype=int32)

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [159]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

central_merged = central_toronto

# merge neighborhoods_venues_sorted with central_toronto to add latitude/longitude and postal code for each neighborhood
central_merged = central_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

central_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Swim School,Business Service,Bus Line,Yoga Studio,Food & Drink Shop,Donut Shop,Farmers Market,Fast Food Restaurant,Fried Chicken Joint
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936,3,Home Service,Garden,Vietnamese Restaurant,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop,Gas Station,Fried Chicken Joint,Food & Drink Shop
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Department Store,Hotel,Park,Sandwich Place,Breakfast Spot,Food & Drink Shop,Fast Food Restaurant,Diner,Donut Shop,Farmers Market
3,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307,2,Jewelry Store,Trail,Park,Sushi Restaurant,Food & Drink Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Yoga Studio
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1,Clothing Store,Coffee Shop,Café,Gym / Fitness Center,Fast Food Restaurant,Diner,Metro Station,Mexican Restaurant,Miscellaneous Shop,Park


### Visualizing the resulting clusters

In [151]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(central_merged['Latitude'], central_merged['Longitude'], central_merged['Neighborhood'], central_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [152]:
central_merged.loc[central_merged['Cluster Labels'] == 0, central_merged.columns[[1] + list(range(5, central_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0,Park,Swim School,Business Service,Bus Line,Yoga Studio,Food & Drink Shop,Donut Shop,Farmers Market,Fast Food Restaurant,Fried Chicken Joint


#### Cluster 2

In [153]:
central_merged.loc[central_merged['Cluster Labels'] == 1, central_merged.columns[[1] + list(range(5, central_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Central Toronto,1,Department Store,Hotel,Park,Sandwich Place,Breakfast Spot,Food & Drink Shop,Fast Food Restaurant,Diner,Donut Shop,Farmers Market
4,Central Toronto,1,Clothing Store,Coffee Shop,Café,Gym / Fitness Center,Fast Food Restaurant,Diner,Metro Station,Mexican Restaurant,Miscellaneous Shop,Park
5,Central Toronto,1,Café,Sandwich Place,Coffee Shop,Pizza Place,Park,BBQ Joint,History Museum,Pharmacy,Indian Restaurant,Burger Joint
6,Central Toronto,1,Pizza Place,Dessert Shop,Sandwich Place,Gym,Sushi Restaurant,Italian Restaurant,Coffee Shop,Café,Pharmacy,Gourmet Shop
8,Central Toronto,1,Coffee Shop,American Restaurant,Restaurant,Vietnamese Restaurant,Fried Chicken Joint,Light Rail Station,Liquor Store,Pub,Pizza Place,Supermarket


#### Cluster 3

In [154]:
central_merged.loc[central_merged['Cluster Labels'] == 2, central_merged.columns[[1] + list(range(5, central_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Central Toronto,2,Jewelry Store,Trail,Park,Sushi Restaurant,Food & Drink Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Yoga Studio


#### Cluster 4

In [155]:
central_merged.loc[central_merged['Cluster Labels'] == 3, central_merged.columns[[1] + list(range(5, central_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Central Toronto,3,Home Service,Garden,Vietnamese Restaurant,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop,Gas Station,Fried Chicken Joint,Food & Drink Shop


#### Cluster 5

In [156]:
central_merged.loc[central_merged['Cluster Labels'] == 4, central_merged.columns[[1] + list(range(5, central_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Central Toronto,4,Tennis Court,Summer Camp,Yoga Studio,Department Store,Gym,Greek Restaurant,Gourmet Shop,Gas Station,Garden,Fried Chicken Joint
